# BERTを用いたスロットフィリング

## 準備

### パッケージのインストール

In [1]:
!pip install -q transformers==4.10.2 seqeval==1.2.2 datasets==1.12.1 numpy==1.19.5

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 270 kB 47.8 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 3.3 MB 33.5 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 636 kB 43.0 MB/s 
     |████████████████████████████████| 243 kB 46.2 MB/s 
     |████████████████████████████████| 1.3 MB 39.5 MB/s 
     |████████████████████████████████| 119 kB 48.0 MB/s 
     |████████████████████████████████| 294 kB 51.4 MB/s 
     |████████████████████████████████| 142 kB 42.9 MB/s 


### インポート

In [2]:
import json
from itertools import chain

import numpy as np
import torch
from datasets import load_dataset, load_metric
from seqeval.metrics import classification_report
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

### データのアップロード

データセットをアップロードします。ノートブックと同じ階層にDataフォルダがあり、その下にsnipsフォルダがあるので、学習・検証用データセットをアップロードしましょう。Colabでない場合は、データセットを読み込むときに正しいパスを指定します。

In [3]:
from google.colab import files

uploaded = files.upload()

Saving train_PlayMusic_full.json to train_PlayMusic_full.json
Saving validate_PlayMusic.json to validate_PlayMusic.json


### データの読み込み

In [4]:
def load_data(filename):
    with open(filename, encoding="iso-8859-2") as f:
        datalist = json.load(f)
    x, y = [], []
    for data in datalist["PlayMusic"]:
        sent = []
        tags = []
        for phrase in data["data"]:
            words = phrase["text"].strip().split()
            if "entity" in phrase:
                label = phrase["entity"]
                labels = [f"B-{label}"] + [f"I-{label}"] * (len(words) - 1)
            else:
                labels = ["O"] * len(words)
            sent.extend(words)
            tags.extend(labels)
        x.append(sent)
        y.append(tags)
    return x, y

In [5]:
train_file = "train_PlayMusic_full.json"
test_file = "validate_PlayMusic.json"

x_train, y_train = load_data(train_file)
x_test, y_test = load_data(test_file)

## 前処理

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
unique_tags = list(set(chain(*y_train)))
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [8]:
train_encodings = tokenizer(
    x_train,
    is_split_into_words=True,
    return_offsets_mapping=True,
    padding=True,
    truncation=True
)
test_encodings = tokenizer(
    x_test,
    is_split_into_words=True,
    return_offsets_mapping=True,
    padding=True,
    truncation=True
)

In [9]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(y_train, train_encodings)
test_labels = encode_tags(y_test, test_encodings)

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
test_encodings.pop("offset_mapping")
train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)

## モデルの学習

「BERTを用いた固有表現認識」のノートブックと同様に、モデルを学習していきましょう。

In [11]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(unique_tags)
)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [12]:
batch_size = 16
args = TrainingArguments(
    "ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
)

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [14]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [15]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [16]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [17]:
trainer.train()

***** Running training *****
  Num examples = 2000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:290: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.361724,0.767857,0.834951,0.800000,0.898817
2,No log,0.239069,0.832558,0.868932,0.850356,0.932983
3,No log,0.264992,0.861905,0.878641,0.870192,0.931669
4,0.319900,0.242983,0.858491,0.883495,0.870813,0.938239
5,0.319900,0.283874,0.876190,0.893204,0.884615,0.943495
6,0.319900,0.305204,0.872642,0.898058,0.885167,0.942181
7,0.319900,0.310534,0.885167,0.898058,0.891566,0.939553
8,0.030100,0.336559,0.898551,0.902913,0.900726,0.939553
9,0.030100,0.333475,0.885167,0.898058,0.891566,0.939553
10,0.030100,0.342922,0.889423,0.898058,0.893720,0.938239


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
Saving model checkpoint to ner/checkpoint-500
Configuration saved in ner/checkpoint-500/config.json
Model weights saved in ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in ner/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:290: UserWarning: To copy construct from a ten

TrainOutput(global_step=1250, training_loss=0.1421953010559082, metrics={'train_runtime': 337.3885, 'train_samples_per_second': 59.279, 'train_steps_per_second': 3.705, 'total_flos': 285834669120000.0, 'train_loss': 0.1421953010559082, 'epoch': 10.0})

## Reference

- [Fine-tuning with custom datasets](https://huggingface.co/transformers/custom_datasets.html)